In [95]:
# Import necessary libraries
import cv2
import numpy as np
import os
import mediapipe as mp
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization, Bidirectional, Conv1D, MaxPooling1D, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from tensorflow.keras.callbacks import TensorBoard
import random
import tensorflow as tf


In [96]:
# Set random seeds for reproducibility
np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

In [97]:
mp_holistic = mp.solutions.holistic  # Holistic model
mp_drawing = mp.solutions.drawing_utils  # Drawing utilities

def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Color conversion
    image.flags.writeable = False
    results = model.process(image)  # Make prediction
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)  # Color conversion
    return image, results

def draw_styled_landmarks(image, results):
    # Draw face connections
    mp_drawing.draw_landmarks(
        image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS,
        mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
        mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
    )
    # Draw pose connections
    mp_drawing.draw_landmarks(
        image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
        mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
        mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
    )
    # Draw left hand connections
    mp_drawing.draw_landmarks(
        image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
        mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
        mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
    )
    # Draw right hand connections
    mp_drawing.draw_landmarks(
        image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
        mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
        mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
    )

def extract_key_points(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]
                    ).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]
                    ).flatten() if results.face_landmarks else np.zeros(468*3)
    left_hand = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]
                         ).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    right_hand = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]
                          ).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, left_hand, right_hand])


In [98]:
# Path for exported data
DATA_PATH = os.path.join('AUSLAN_Data')

# Actions that we try to detect
actions = np.array(['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z'])

# actions = np.array(['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P'])

# actions = np.array(['R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z'])

# actions = np.array(['G', 'H'])
# actions = np.array(['J'])

# Number of sequences (videos)
no_sequences = 50

# Length of each sequence (frames)
sequence_length = 30

# Label mapping
label_map = {label: num for num, label in enumerate(actions)}

In [99]:
print(label_map)

{'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6, 'H': 7, 'I': 8, 'J': 9, 'K': 10, 'L': 11, 'M': 12, 'N': 13, 'O': 14, 'P': 15, 'Q': 16, 'R': 17, 'S': 18, 'T': 19, 'U': 20, 'V': 21, 'W': 22, 'X': 23, 'Y': 24, 'Z': 25}


### 1. Data Preprocessing
- Normalization and Centering

In [100]:
def normalize_keypoints(sequence):
    # Reshape sequence to (frames, keypoints, coordinates)
    sequence = sequence.reshape(sequence.shape[0], -1, 3)
    # Compute mean and std for each coordinate axis
    mean = np.mean(sequence, axis=1, keepdims=True)
    std = np.std(sequence, axis=1, keepdims=True) + 1e-6  # Add epsilon to avoid division by zero
    # Normalize
    normalized_sequence = (sequence - mean) / std
    return normalized_sequence.reshape(sequence.shape[0], -1)


- Load and Normalize Data

In [101]:
sequences, labels = [], []
for action in actions:
    for sequence_num in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence_num), f"{frame_num}.npy"))
            window.append(res)
        window = normalize_keypoints(np.array(window))
        sequences.append(window)
        labels.append(label_map[action])

# Convert to numpy arrays
X = np.array(sequences)
y = to_categorical(labels).astype(int)


### 2. Data Augmentation
- Enhanced Augmentation Functions

In [102]:
# Noise Injection
def add_noise(sequence, noise_level=0.05):
    noise = np.random.normal(0, noise_level, sequence.shape)
    return sequence + noise

# Scaling
def scale_sequence(sequence, scale_factor=0.1):
    scaling = np.random.normal(1.0, scale_factor, (sequence.shape[0], 1))
    return sequence * scaling

# Time Warping
def time_warp(sequence, sigma=0.2):
    from scipy.interpolate import CubicSpline
    num_frames = sequence.shape[0]
    random_warp = np.random.normal(loc=1.0, scale=sigma, size=num_frames)
    cumulative_warp = np.cumsum(random_warp)
    cumulative_warp = (cumulative_warp - cumulative_warp.min()) / (cumulative_warp.max() - cumulative_warp.min()) * (num_frames - 1)
    cs = CubicSpline(np.arange(num_frames), sequence, axis=0)
    warped_sequence = cs(cumulative_warp)
    return warped_sequence

# Function to randomly augment a sequence
def augment_sequence(sequence):
    augmented_sequence = sequence.copy()
    if np.random.rand() < 0.5:
        augmented_sequence = add_noise(augmented_sequence)
    if np.random.rand() < 0.5:
        augmented_sequence = scale_sequence(augmented_sequence)
    if np.random.rand() < 0.5:
        augmented_sequence = time_warp(augmented_sequence)
    return augmented_sequence


- Apply Augmentation

In [103]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True)

# Augment training data
augmented_sequences = []
augmented_labels = []

for seq, label in zip(X_train, y_train):
    augmented_seq = augment_sequence(seq)
    augmented_sequences.append(augmented_seq)
    augmented_labels.append(label)

# Convert augmented data to numpy arrays
augmented_sequences = np.array(augmented_sequences)
augmented_labels = np.array(augmented_labels)

# Combine original and augmented data
X_train_augmented = np.concatenate((X_train, augmented_sequences), axis=0)
y_train_augmented = np.concatenate((y_train, augmented_labels), axis=0)

# Shuffle the augmented training data
X_train_augmented, y_train_augmented = shuffle(X_train_augmented, y_train_augmented)


### 3. Model Definition

In [163]:
# from tensorflow.keras.regularizers import l2

# model = Sequential()
# model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(sequence_length, X.shape[2])))
# model.add(LSTM(128, return_sequences=True, activation='relu'))
# model.add(LSTM(64, activation='relu'))
# model.add(Dense(64, activation='relu'))
# model.add(Dense(len(actions), activation='softmax'))

# optimizer = Adam(learning_rate=1e-4)
# model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['categorical_accuracy'])



from tensorflow.keras.regularizers import l2

model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(sequence_length, X.shape[2])))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(LSTM(64, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(64, activation='relu'))
model.add(Dense(len(actions), activation='softmax'))

optimizer = Adam(learning_rate=1e-4)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['categorical_accuracy'])



c:\Users\khobr\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


### 4. Training with Callbacks

In [142]:
# Compute class weights (if needed)
from sklearn.utils.class_weight import compute_class_weight

labels_flat = np.argmax(y_train_augmented, axis=1)
class_weights = compute_class_weight('balanced', classes=np.unique(labels_flat), y=labels_flat)
class_weights = dict(enumerate(class_weights))

# Define callbacks
callbacks = [
    ModelCheckpoint('baseline_model_tunning3.keras', save_best_only=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5),
    EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
]




In [143]:
# Train the model
history = model.fit(
    X_train_augmented, y_train_augmented,
    epochs=40,
    batch_size=32,
    validation_data=(X_test, y_test),
    callbacks=callbacks,
    class_weight=class_weights
)

Epoch 1/40
65/65 ━━━━━━━━━━━━━━━━━━━━ 26s 136ms/step - categorical_accuracy: 0.0638 - loss: 3.3218 - val_categorical_accuracy: 0.0769 - val_loss: 3.2272 - learning_rate: 1.0000e-04
Epoch 2/40
65/65 ━━━━━━━━━━━━━━━━━━━━ 6s 91ms/step - categorical_accuracy: 0.0972 - loss: 3.1248 - val_categorical_accuracy: 0.0808 - val_loss: 3.2469 - learning_rate: 1.0000e-04
Epoch 3/40
65/65 ━━━━━━━━━━━━━━━━━━━━ 6s 99ms/step - categorical_accuracy: 0.1721 - loss: 2.8435 - val_categorical_accuracy: 0.1038 - val_loss: 3.2113 - learning_rate: 1.0000e-04
Epoch 4/40
65/65 ━━━━━━━━━━━━━━━━━━━━ 7s 102ms/step - categorical_accuracy: 0.2752 - loss: 2.6334 - val_categorical_accuracy: 0.1038 - val_loss: 3.1731 - learning_rate: 1.0000e-04
Epoch 5/40
65/65 ━━━━━━━━━━━━━━━━━━━━ 6s 98ms/step - categorical_accuracy: 0.3506 - loss: 2.4286 - val_categorical_accuracy: 0.1923 - val_loss: 2.9891 - learning_rate: 1.0000e-04
Epoch 6/40
65/65 ━━━━━━━━━━━━━━━━━━━━ 6s 100ms/step - categorical_accuracy: 0.4257 - loss: 2.2167 - va

### 5. Evaluation

In [164]:
# Load the best model
model.load_weights('baseline_model_tunning2.keras')

c:\Users\khobr\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\saving\saving_lib.py:415: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 40 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [165]:
# Evaluate on test set
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}')
print(f'Test Accuracy: {accuracy}')

9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 49ms/step - categorical_accuracy: 0.9875 - loss: 0.0409
Test Loss: 0.047065407037734985
Test Accuracy: 0.9846153855323792


In [166]:
# Confusion Matrix
from sklearn.metrics import multilabel_confusion_matrix, classification_report

y_pred = model.predict(X_test)
y_pred_decoded = np.argmax(y_pred, axis=1)
y_test_decoded = np.argmax(y_test, axis=1)

print(classification_report(y_test_decoded, y_pred_decoded, target_names=actions))

9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 115ms/step
              precision    recall  f1-score   support

           A       1.00      0.88      0.93         8
           B       0.92      1.00      0.96        11
           C       0.89      1.00      0.94         8
           D       1.00      0.90      0.95        10
           E       1.00      1.00      1.00        13
           F       1.00      0.92      0.96        13
           G       1.00      1.00      1.00         5
           H       1.00      1.00      1.00        12
           I       1.00      1.00      1.00        11
           J       0.92      1.00      0.96        12
           K       1.00      1.00      1.00         8
           L       1.00      1.00      1.00        11
           M       1.00      1.00      1.00        15
           N       1.00      1.00      1.00        12
           O       1.00      1.00      1.00        10
           P       0.83      1.00      0.91         5
           Q       1.00      1.00      1.0

### Real Time Detection Begin:

In [159]:
# Define your normalization function (from your training code)
def normalize_keypoints(sequence):
    # Reshape sequence to (frames, keypoints, coordinates)
    sequence = sequence.reshape(sequence.shape[0], -1, 3)
    # Compute mean and std for each coordinate axis
    mean = np.mean(sequence, axis=1, keepdims=True)
    std = np.std(sequence, axis=1, keepdims=True) + 1e-6  # Add epsilon to avoid division by zero
    # Normalize
    normalized_sequence = (sequence - mean) / std
    return normalized_sequence.reshape(sequence.shape[0], -1)

# Define the preprocess_frame function
def preprocess_frame(results):
    # Extract keypoints
    keypoints = extract_key_points(results)
    # Normalize keypoints
    keypoints = normalize_keypoints(np.expand_dims(keypoints, axis=0))
    return keypoints[0]

In [160]:
colors = [
    (255, 0, 0),      # A
    (0, 255, 0),      # B
    (0, 0, 255),      # C
    (255, 255, 0),    # D
    (0, 255, 255),    # E
    (255, 0, 255),    # F
    (190, 125, 0),    # G
    (0, 190, 125),    # H
    (190, 0, 125),    # I
    (25, 185, 0),     # J
    (0, 25, 185),     # K
    (185, 0, 25),     # L
    (100, 0, 100),    # M
    (0, 100, 100),    # N
    (123, 123, 0),    # O
    (255, 165, 0),    # P
    (75, 0, 130),     # Q
    (255, 20, 147),   # R
    (0, 128, 0),      # S
    (128, 0, 128),    # T
    (0, 0, 128),      # U
    (128, 128, 0),    # V
    (0, 100, 200),    # W
    (28, 20, 50),   # X
    (85, 100, 128),   # Y
    (70, 75, 75)   # Z
]


In [161]:

def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    
    # Get the dimensions of the window
    frame_height, frame_width, _ = output_frame.shape
    
    # Set configuration for dynamic layout
    left_column_letters = 11  # Number of letters on the left side
    top_row_letters = 5       # Number of letters on the top side
    label_height = 40         # Vertical spacing for left and right
    label_width = 120         # Maximum width of rectangles for probability bars
    top_spacing = 40          # Adjustable spacing for top row (set this to control top padding)
    top_letter_spacing = 5    # Adjustable spacing between top letters

    # Iterate over the results and position letters accordingly
    for num, prob in enumerate(res):
        if num < left_column_letters:
            # Left-side placement (first 11 letters)
            x_offset = 0
            y_position = top_spacing + num * label_height
            # Probability bars increasing to the right (normal)
            cv2.rectangle(output_frame, (x_offset, y_position), 
                          (x_offset + int(prob * 100), y_position + 30), 
                          colors[num % len(colors)], -1)
            cv2.putText(output_frame, actions[num], (x_offset, y_position + 25), 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

        elif num < left_column_letters + top_row_letters:
            # Top-side placement (next 5 letters)
            x_offset = 30 + (num - left_column_letters) * (label_width + top_letter_spacing)
            y_position = top_spacing  # Control the vertical position of the top letters
            # Dynamic width based on probability
            cv2.rectangle(output_frame, (x_offset, y_position), 
                          (x_offset + int(prob * 100), y_position + 30), 
                          colors[num % len(colors)], -1)
            cv2.putText(output_frame, actions[num], (x_offset, y_position + 25), 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

        else:
            # Right-side placement (remaining letters)
            x_offset = frame_width - label_width  # Adjust the starting point to be near the edge
            y_position = top_spacing + (num - left_column_letters - top_row_letters) * label_height
            # Probability bars increasing to the left (opposite direction)
            bar_start = x_offset + label_width - int(prob * 100)
            cv2.rectangle(output_frame, 
                          (bar_start, y_position),  # Shift to the left as bar increases
                          (x_offset + label_width, y_position + 30), 
                          colors[num % len(colors)], -1)
            # Position text exactly where the bar starts
            cv2.putText(output_frame, actions[num], 
                        (bar_start - 20, y_position + 25),  # Adjusted to align with bar start
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
    
    return output_frame




In [167]:
# New Detection variables
sequence = []
sentence = []
predictions = []
threshold = 0.7

# Load your trained model
# model = best_model  # Make sure 'best_model' is defined or load your model accordingly

cap = cv2.VideoCapture(0)

# Access MediaPipe model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        # Read the feed
        ret, frame = cap.read()

        # Flip the frame horizontally if needed
        # frame = cv2.flip(frame, 1)

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)

        # Draw landmarks
        draw_styled_landmarks(image, results)

        # Preprocess the frame
        keypoints = preprocess_frame(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]  # Keep only the last 30 frames

        # Once we have 30 frames, make a prediction
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))

            # Visualize results if the prediction is consistent
            if np.unique(predictions[-10:])[0] == np.argmax(res):
                if res[np.argmax(res)] > threshold:
                    if len(sentence) > 0:
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])

                if len(sentence) > 5:
                    sentence = sentence[-5:]

            # Show probability visualization
            image = prob_viz(res, actions, image, colors)

        # Display the result
        cv2.rectangle(image, (0, 0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

        # Show to the screen
        cv2.imshow('Frame', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

c:\Users\khobr\AppData\Local\Programs\Python\Python310\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

### Video Recording while real time detection

In [170]:
# New Detection variables
sequence = []
sentence = []
predictions = []
threshold = 0.7

# Load your trained model

###############################################################
# Initialize video capture
cap = cv2.VideoCapture(0)

# Get frame width and height (required for VideoWriter)
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Define the codec and create VideoWriter object
# You can change 'output.avi' to your desired filename and path
fourcc = cv2.VideoWriter_fourcc(*'XVID')  # You can use other codecs like 'MJPG', 'XVID', 'MP4V', etc.
out = cv2.VideoWriter('output.avi', fourcc, 20.0, (frame_width, frame_height))
####################################################################

# Access MediaPipe model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        # Read the feed
        ret, frame = cap.read()

        if not ret:
            print("Ignoring empty camera frame.")
            break

        # Flip the frame horizontally if needed
        # frame = cv2.flip(frame, 1)

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        # print(results)  # Uncomment if you want to see the detection results

        # Draw landmarks
        draw_styled_landmarks(image, results)

        # Preprocess the frame
        keypoints = preprocess_frame(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]  # Keep only the last 30 frames

        # Once we have 30 frames, make a prediction
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))

            # Visualize results if the prediction is consistent
            if np.unique(predictions[-10:])[0] == np.argmax(res):
                if res[np.argmax(res)] > threshold:
                    if len(sentence) > 0:
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])

                if len(sentence) > 5:
                    sentence = sentence[-5:]

            # Show probability visualization
            image = prob_viz(res, actions, image, colors)

        # Display the result
        cv2.rectangle(image, (0, 0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

        # Write the frame into the file 'output.avi'
        out.write(image)

        # Show to the screen
        cv2.imshow('Frame', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    # Release everything when job is finished
    cap.release()
    out.release()
    cv2.destroyAllWindows()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
S
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
C
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
S
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
S
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
S
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
S
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
S
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
S
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
S
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
S
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
S
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
S
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
S
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
S
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
S
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
C
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
S
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
C
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
S
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
C
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
S
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
C
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
S
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
C
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
C
